# Tensorflow MNIST Pixel Threshold demo

## Setup

Below we import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected,

In [ ]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple

# Filter out warning messages
warnings.filterwarnings("ignore")

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:30080"

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

Check that the Makefile works in your environment by executing the `bash` code block below,

In [ ]:
%%bash

# Running this will just list the available rules defined in the demo's Makefile.
make

This demo has the same prerequisites as the **Tensorflow MNIST Classifier** example, so we follow the same setup pattern of downloading and organizing the MNIST dataset, followed by deploying the lab architecture microservices:

In [ ]:
%%bash

# If you have already run this command, you should just see the message:
#
#     make: Nothing to be done for `data'.

make data

In [ ]:
%%bash

# Deploy the lab microservices declared in docker-compose.yml
make services

The user is referred to the notebook in the [tensorflow-mnist-classifier](../tensorflow-mnist-classifier) example folder for additional details.

## Submit and run jobs

Like in the **Tensorflow MNIST Classifier** example, the jobs that we will be running are implemented in the Python source files under `src/`, which will be executed using the entrypoints defined in the `MLproject` file.
These files need to be packaged into an archive before they can be uploaded to the API as part of the job submission.
To package the files, run `make workflows`,

In [ ]:
%%bash

# Create the workflows.tar.gz file
make workflows

To interact with the lab API, we use the client defined in the `utils.py` file,

In [ ]:
restapi_client = utils.SecuringAIClient(address=RESTAPI_API_BASE)

We also need to register an experiment under which to collect our job runs.
We will name the experiment `"mnist"`.
The code below checks if the relevant experiment named `"mnist"` exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [ ]:
response_experiment = restapi_client.get_experiment_by_name(name="mnist")

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name="mnist")

response_experiment

Next, we need to train a neural network architecture on the MNIST dataset.
We will be using the LeNet-5 architecture for this demo.
Depending on the specs of your computer, training the model on a CPU can take 10-20 minutes or longer to complete.
If you are fortunate enough to have access to a dedicated GPU, then the training time will be much shorter.
So that we do not start this code by accident, we add a guard around the code that checks whether a model is registered under the name `mnist_le_net`.
If it doesn't then training proceeds, otherwise the code will be skipped.
If you haven't trained the model yet, do so now,

In [ ]:
registered_models = [x.name for x in MlflowClient().list_registered_models()]

if "mnist_le_net" not in registered_models:
    response_le_net_train = restapi_client.submit_job(
        workflows_file=WORKFLOWS_TAR_GZ,
        experiment_name="mnist",
        entry_point="train",
        entry_point_kwargs=" ".join([
            "-P register_model=True",
            "-P model_architecture=le_net",
        ]),
    )

    print("Training job for LeNet-5 neural network submitted")
    print("")
    pprint.pprint(response_le_net_train)

You can monitor the status of the job by querying the endpoint using the code below, which again will not run if a trained model is already registered.

In [ ]:
def job_id_or_none():
    try:
        return response_le_net_train.get("jobId")

    except NameError:
        pass
    
    return None


if job_id_or_none() is not None:
    response_le_net_train = restapi_client.get_job_by_id(
        job_id_or_none()
    )

    pprint.pprint(response_le_net_train)

With the LeNet-5 model properly trained, it is now time to apply the Pixel Threshold evasion attack to generate adversarial images to try and fool the classifier.
The success of the evasion attack is then measured by applying standard machine learning metrics to the predictions made on the generated adversarial images.
Below we declare a dependency on the training job so that we can queue up this job to start immediately after training is complete.
The message "Pixel Threshold attack (LeNet-5 architecture) job submitted" will display upon successful submission.

In [ ]:
response_pt_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name="mnist",
    entry_point="pt",
    entry_point_kwargs=" ".join(
        ["-P model=mnist_le_net/1", "-P model_architecture=le_net", "-P th=1"]
    ),
    depends_on=job_id_or_none(),
)

print("Pixel Threshold attack (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_pt_le_net)
print("")

We can poll the status of the job using the code below.
We should see the status of the jobs shift from "queued" to "started" and eventually become "finished".

In [ ]:
response_pt_le_net = restapi_client.get_job_by_id(
    response_pt_le_net["jobId"]
)

pprint.pprint(response_pt_le_net)

## Cleanup

To clean up, you simply need to shut down the services, which can be done using the following Makefile rule,

In [ ]:
%%bash

# Stop and remove all microservices
make teardown